In [1]:
## Packeges
import pandas as pd
import numpy as np

In [2]:
rent = pd.read_csv('Rent_Contracts.csv')

In [12]:
staging_db = rent.copy()
staging_db.head()

,contract_id,contract_reg_type_id,contract_reg_type_ar,contract_reg_type_en,contract_start_date,contract_end_date,contract_amount,annual_amount,no_of_prop,line_number,...,actual_area,nearest_landmark_ar,nearest_landmark_en,nearest_metro_ar,nearest_metro_en,nearest_mall_ar,nearest_mall_en,tenant_type_id,tenant_type_ar,tenant_type_en
0,CRT1012981266,1,جديد,New,07-04-2019,06-04-2020,85000,85000,1,1,...,140.0,وسط مدينة دبي,Downtown Dubai,محطة مترو بوج خليفة دبي مول,Buj Khalifa Dubai Mall Metro Station,مول دبي,Dubai Mall,2.0,جهة,Authority
1,CRT1012983196,1,جديد,New,20-04-2019,19-04-2020,110000,110000,1,1,...,734.0,أكاديمية المدينة الرياضية للسباحة,Sports City Swimming Academy,محطة مترو النخيل,Nakheel Metro Station,مارينا مول,Marina Mall,1.0,شخص,Person
2,CRT1012984226,1,جديد,New,11-04-2019,10-04-2020,100000,100000,1,1,...,324.0,دورة دبي للدراجات,Dubai Cycling Course,NaN,NaN,NaN,NaN,1.0,شخص,Person
3,CRT1012984996,2,تجديد,Renew,18-03-2019,17-03-2020,150000,150000,1,1,...,405.0,موتور سيتي,Motor City,NaN,NaN,NaN,NaN,1.0,شخص,Person
4,CRT1012986616,1,جديد,New,15-04-2019,14-04-2020,95000,95000,1,1,...,103.0,برج العرب,Burj Al Arab,مساكن شاطئ جميرا,Jumeirah Beach Residency,مارينا مول,Marina Mall,1.0,شخص,Person


In [13]:
#DimContractType Table
DimContractType = staging_db[['contract_reg_type_id','contract_reg_type_ar','contract_reg_type_en']]
staging_db.drop(['contract_reg_type_ar','contract_reg_type_en'],axis=1,inplace = True)
DimContractType = DimContractType.drop_duplicates().reset_index(drop=True)
DimContractType

,contract_reg_type_id,contract_reg_type_ar,contract_reg_type_en
0,1,جديد,New
1,2,تجديد,Renew


In [5]:
## Cleaning contract_start_date and contract_end_date columns in simple way "Each value out 2050 is mistake" 
from pandas._libs import OutOfBoundsDatetime


temp = staging_db[['contract_start_date','contract_end_date']]
# temp = temp.drop_duplicates()

ind_start = []
ind_end = []
print("The Faults in the start date:")
for i,y in zip(temp['contract_start_date'],range(len(temp))):
    try:
        if pd.to_datetime(i,format = 'mixed').year > 2050:
            print(i,y)
            ind_start.append(y)
    except OutOfBoundsDatetime:
        print(i,y)
        ind_end.append(y)
print("The Faults in the end date:")
for i,y in zip(temp['contract_end_date'],range(len(temp))):
    try:
        if pd.to_datetime(i,format = 'mixed').year > 2050:
            print(i,y)
            ind_start.append(y)
    except OutOfBoundsDatetime:
        print(i,y)
        ind_end.append(y)

The Faults in the start date:
05-07-2109 1888062
05-06-2107 5865415
05-06-2108 6625586
The Faults in the end date:
04-07-2110 1888062
31-03-3013 5776276
04-06-2108 5865415
04-07-2109 6625586


In [6]:
#Fixing the Mistake in Dates Columns
#actually i did this task manually because i watn to make my code faster "the data set so big"


#Mistakes in Start Col
# 05-07-2109 75710
# 05-06-2107 156271
# 05-06-2108 170662
#05-07-2109 1888062
#05-06-2107 5865415
#05-06-2108 6625586
staging_db['contract_start_date'].iloc[75710] = "04-07-2011"
staging_db['contract_start_date'].iloc[156271] = "04-06-2018"
staging_db['contract_start_date'].iloc[170662] = "04-07-2019"
staging_db['contract_start_date'].iloc[1888062] = "05-07-2019"
staging_db['contract_start_date'].iloc[5865415] = "05-06-2017"
staging_db['contract_start_date'].iloc[6625586] = "05-06-2018"


# Mistakes in End 
# 04-07-2110 75710
# 31-03-3013 154695
# 04-06-2108 156271
# 04-07-2109 170662
# 04-07-2110 1888062
# 31-03-3013 5776276
# 04-06-2108 5865415
# 04-07-2109 6625586
staging_db['contract_end_date'].iloc[75710] = "04-07-2011"
staging_db['contract_end_date'].iloc[154695] = "31-03-2013"
staging_db['contract_end_date'].iloc[156271] = "04-06-2018"
staging_db['contract_end_date'].iloc[170662] = "04-07-2019"
staging_db['contract_end_date'].iloc[1888062] = "04-07-2011"
staging_db['contract_end_date'].iloc[5776276] = "31-03-2013"
staging_db['contract_end_date'].iloc[5865415] = "04-06-2018"
staging_db['contract_end_date'].iloc[6625586] = "04-07-2019"




C:\Users\hussam\AppData\Local\Temp\ipykernel_31344\565747486.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_db['contract_start_date'].iloc[75710] = "04-07-2011"
C:\Users\hussam\AppData\Local\Temp\ipykernel_31344\565747486.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_db['contract_start_date'].iloc[156271] = "04-06-2018"
C:\Users\hussam\AppData\Local\Temp\ipykernel_31344\565747486.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [7]:
# DimDate Table
Dim_date = staging_db[['contract_start_date','contract_end_date']]
Dim_date['contract_start_date'] = Dim_date['contract_start_date']
Dim_date['contract_end_date'] = Dim_date['contract_end_date']



contract_days_list = []

Dim_date = Dim_date.drop_duplicates().reset_index(drop=True)
for i in range(len(Dim_date)):
    temp = pd.to_datetime(Dim_date['contract_end_date'].iloc[i],format='mixed') - pd.to_datetime(Dim_date['contract_start_date'].iloc[i],format='mixed')
    contract_days_list.append(temp.days)
    
Dim_date['contract_days'] = pd.Series(contract_days_list)
Dim_date = Dim_date.sort_values(by = 'contract_days', ascending=True).reset_index(drop=True)
DimDate = Dim_date
# staging_db = staging_db.drop(['contract_start_date', 'contract_end_date'],axis=1)
DimDate.head(15)


C:\Users\hussam\AppData\Local\Temp\ipykernel_31344\2725254335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_date['contract_start_date'] = Dim_date['contract_start_date']
C:\Users\hussam\AppData\Local\Temp\ipykernel_31344\2725254335.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_date['contract_end_date'] = Dim_date['contract_end_date']


,contract_start_date,contract_end_date,contract_days
0,05-07-2019,04-07-2011,-2952
1,26-07-2020,31-03-2013,-2674
2,12-03-2012,01-06-2012,-332
3,12-01-2020,01-05-2020,-331
4,12-07-2015,01-10-2015,-331
5,12-03-2021,01-07-2021,-330
6,12-07-2022,01-11-2022,-330
7,12-04-2023,01-11-2023,-327
8,12-01-2024,01-10-2024,-326
9,12-02-2022,01-10-2022,-326


In [8]:
#Drop the minues Contract_days values
DimDate = DimDate[DimDate['contract_days']>0]
DimDate.reset_index(inplace=True)
DimDate['contract_date_Id'] = pd.Series(range(len(DimDate)))
DimDate = DimDate[['contract_date_Id', 'contract_start_date', 'contract_end_date', 'contract_days']]
DimDate

C:\Users\hussam\AppData\Local\Temp\ipykernel_31344\1269525707.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DimDate['contract_date_Id'] = pd.Series(range(len(DimDate)))


,contract_date_Id,contract_start_date,contract_end_date,contract_days
0,0,04-05-2013,04-06-2013,1
1,1,29-11-2015,30-11-2015,1
2,2,01-02-2019,01-03-2019,1
3,3,31-12-2011,01-01-2012,1
4,4,01-02-2017,01-03-2017,1
...,...,...,...,...
164678,164678,01-01-2012,31-12-2026,5478
164679,164679,01-03-2014,15-02-2029,5522
164680,164680,14-04-2010,27-09-2025,5645
164681,164681,01-04-2006,31-03-2028,8122


### ⚠ I will look up the staging from DimDate using look up tool in SSIS So I will Keep Start and End Date columns for right now ⚠





.

In [14]:
#DimEjaribuspropertytype Table
DimEjaribuspropertytype = staging_db[['ejari_bus_property_type_id','ejari_bus_property_type_ar','ejari_bus_property_type_en']]
DimEjaribuspropertytype = DimEjaribuspropertytype.drop_duplicates().reset_index(drop=True)
staging_db.drop(['ejari_bus_property_type_ar','ejari_bus_property_type_en'],axis=1,inplace = True)
DimEjaribuspropertytype

,ejari_bus_property_type_id,ejari_bus_property_type_ar,ejari_bus_property_type_en
0,2,وحدة,Unit
1,4,فيلا,Villa
2,1,مبنى,Building
3,0,أرض,Land


In [15]:
#DimEjaripropertytype Table
DimEjaripropertytype = staging_db[['ejari_property_type_id','ejari_property_type_en','ejari_property_type_ar']]
DimEjaripropertytype = DimEjaripropertytype.drop_duplicates().reset_index(drop=True)
staging_db.drop(['ejari_property_type_en','ejari_property_type_ar'],axis=1,inplace = True)
DimEjaripropertytype.drop(DimEjaripropertytype[DimEjaripropertytype['ejari_property_type_id'].isnull()].index, axis=0, inplace = True)

DimEjaripropertytype

,ejari_property_type_id,ejari_property_type_en,ejari_property_type_ar
0,2.000000e+00,Office,مكتب
1,8.410000e+02,Villa,فيلا
2,8.420000e+02,Flat,شقه
3,1.000000e+00,Shop,محل
4,9.030000e+02,Studio,أستوديو
...,...,...,...
67,1.800000e+01,Shopping Center,مركز تسوق
68,1.808301e+09,swimming pool,حمام السباحة
69,1.348597e+09,Desk,مكت
70,2.900000e+01,University,جامعه


In [16]:
#DimEjaripropertysubtype Table
DimEjaripropertysubtype = staging_db[['ejari_property_sub_type_id','ejari_property_sub_type_en','ejari_property_sub_type_ar']]
DimEjaripropertysubtype = DimEjaripropertysubtype.drop_duplicates().reset_index(drop=True)
staging_db.drop(['ejari_property_sub_type_en','ejari_property_sub_type_ar'],axis=1,inplace = True)
DimEjaripropertysubtype.drop(DimEjaripropertysubtype[DimEjaripropertysubtype['ejari_property_sub_type_id'].isnull()].index, axis=0, inplace = True)

DimEjaripropertysubtype

,ejari_property_sub_type_id,ejari_property_sub_type_en,ejari_property_sub_type_ar
0,4.220000e+02,Office,مكتب
1,2.000000e+00,2 bed rooms+hall,غرفتين و صالة
2,3.000000e+00,3 bed rooms+hall,ثلاثة غرفة و صالة
3,1.000000e+00,1bed room+Hall,غرفة و صالة
4,4.000000e+00,4 bed rooms+hall,اربعة غرف و صالة
...,...,...,...
82,1.240135e+08,OPEN STORAGE SHED,التخزين المفتوحة
83,3.300000e+01,Boardroom,قاعة اجتماعات
84,1.374814e+08,"Internet cafe , Fast Food","مقهى للانترنت ,الوجبات السريعة"
85,1.808300e+09,swimming pool,حمام السباحة


In [17]:
#DimPropertyUsage Table
DimPropertyUsage  = pd.DataFrame(staging_db['property_usage_en'].unique(),columns=['property_usage_en'])
DimPropertyUsage['property_usage_ar']  = pd.DataFrame(staging_db['property_usage_ar'].unique())

DimPropertyUsage['Property_Id'] = pd.Series(range(len(DimPropertyUsage)))
DimPropertyUsage = DimPropertyUsage[['Property_Id', 'property_usage_en', 'property_usage_ar']]
DimPropertyUsage

,Property_Id,property_usage_en,property_usage_ar
0,0,Commercial,تجاري
1,1,Residential,سكني
2,2,NaN,NaN
3,3,Industrial,صناعي
4,4,Educational facility,منشأه تعليميه
5,5,Storage,تخزين
6,6,Health Facility,منشأه صحيه
7,7,Agriculture,زراعة
8,8,Tourist origin,منشأه سياحيه
9,9,Industrial / Commercial,صناعي / تجاري


In [ ]:
# Generate Property Colmn 
temp_mapper = {key:value for key,value in zip(DimPropertyUsage['property_usage_en'],DimPropertyUsage['Property_Id'])}
staging_db['generated_property_Id_PY'] = staging_db['property_usage_en'].map(temp_mapper)
staging_db.drop(['property_usage_en', 'property_usage_ar'],axis=1,inplace=True)



In [20]:
#DimProject
DimProject = staging_db[['project_number', 'project_name_ar', 'project_name_en']]
DimProject =  DimProject.drop_duplicates().reset_index()
DimProject['project_Id'] = pd.Series(range(len(DimProject)))
DimProject = DimProject.sort_values(by = 'project_number')


staging_db = staging_db.drop(['project_name_ar', 'project_name_en'], axis=1)
# staging_db.rename(columns ={'project_number':'project_number_Id'},inplace=True)

DimProject = DimProject[['project_number','project_Id', 'project_name_ar', 'project_name_en']]
DimProject.loc[len(DimProject)] = ['0','88888','غير معروف','Unknown'] # for dealing with unknon project which have 0 Id

DimProject.drop(DimProject[DimProject['project_number'].isnull()]['project_Id'], axis=0, inplace = True)
# project_number_Id


In [21]:
#DimMasterProject Table
DimMasterProject  = pd.DataFrame(staging_db['master_project_en'].unique(),columns=['master_project_en'])
DimMasterProject['master_project_ar']  = pd.DataFrame(staging_db['master_project_ar'].unique())

DimMasterProject['master_property_Id'] = pd.Series(range(len(DimMasterProject)))
DimMasterProject = DimMasterProject[['master_property_Id', 'master_project_en', 'master_project_ar']]
DimMasterProject

,master_property_Id,master_project_en,master_project_ar
0,0,Business Bay,الخليج التجاري
1,1,Jumeirah Village Triangle,قرية جميرا المثلثة
2,2,NaN,NaN
3,3,Arabian Ranches II - PALMA,المرابع العربية 2 - بالما
4,4,Jumeriah Beach Residence - JBR,جميرا بيتش ريزيدنس - الجيه بي آر
...,...,...,...
158,158,Dubai International Financial Center,مركز دبي المالي الدولي
159,159,TECOM Site D,تيكوم سايت د
160,160,Dubai International Academic City,تايجرا تاون
161,161,Tijara Town,المدينة العالمية - المرحلة الثانية


In [22]:
# Generate Master_project_Id Colmn 
temp_mapper = {key:value for key,value in zip(DimMasterProject['master_project_en'],DimMasterProject['master_property_Id'])}
staging_db['generated_master_project_Id_PY'] = staging_db['master_project_en'].map(temp_mapper)
staging_db.drop(['master_project_en', 'master_project_ar'],axis=1,inplace=True)


In [ ]:
# area_id                                                464
# area_name_ar                                  
# area_name_en                             

In [23]:
#DimArea Table
DimArea  = pd.DataFrame(staging_db['area_name_en'].unique(),columns=['area_name_en'])
DimArea['area_name_ar']  = pd.DataFrame(staging_db['area_name_ar'].unique())
DimArea['area_Dim_Id']  = pd.DataFrame(staging_db['area_id'].unique())

DimArea['area_id'] = pd.Series(range(len(DimArea)))
DimArea = DimArea[['area_id', 'area_Dim_Id', 'area_name_en', 'area_name_ar']]
staging_db = staging_db.drop(['area_name_en','area_name_ar'], axis=1)
DimArea 

,area_id,area_Dim_Id,area_name_en,area_name_ar
0,0,526,Business Bay,الخليج التجارى
1,1,442,Al Barsha South Fifth,البرشاء جنوب الخامسة
2,2,506,Al Yelayiss 1,اليلايس 1
3,3,463,Wadi Al Safa 7,وادي الصفا 7
4,4,330,Marsa Dubai,مرسى دبي
...,...,...,...,...
209,209,496,Le Hemaira,الحميرا
210,210,519,Al Layan1,الليان1
211,211,500,Um Esalay,ام السلي
212,212,508,Al Yelayiss 3,اليلايس 3


In [24]:
#DimNearestLandmark Table
DimNearestLandmark  = pd.DataFrame(staging_db['nearest_landmark_en'].unique(),columns=['nearest_landmark_en'])
DimNearestLandmark['nearest_landmark_ar']  = pd.DataFrame(staging_db['nearest_landmark_ar'].unique())

DimNearestLandmark['nearest_landmark_Id'] = pd.Series(range(len(DimNearestLandmark)))
DimNearestLandmark = DimNearestLandmark[['nearest_landmark_Id', 'nearest_landmark_en', 'nearest_landmark_ar']]
DimNearestLandmark

,nearest_landmark_Id,nearest_landmark_en,nearest_landmark_ar
0,0,Downtown Dubai,وسط مدينة دبي
1,1,Sports City Swimming Academy,أكاديمية المدينة الرياضية للسباحة
2,2,Dubai Cycling Course,دورة دبي للدراجات
3,3,Motor City,موتور سيتي
4,4,Burj Al Arab,برج العرب
5,5,NaN,NaN
6,6,Expo 2020 Site,موقع إكسبو 2020
7,7,Burj Khalifa,برج خليفة
8,8,Dubai International Airport,مطار دبي الدولي
9,9,IMG World Adventures,آي إم جي وورلد أدفينتشرز


In [25]:
# Generate Nearest_landmark_Id_PY Colmn 
temp_mapper = {key:value for key,value in zip(DimNearestLandmark['nearest_landmark_en'],DimNearestLandmark['nearest_landmark_Id'])}
staging_db['generated_nearest_landmark_Id_PY'] = staging_db['nearest_landmark_en'].map(temp_mapper)
staging_db.drop(['nearest_landmark_en', 'nearest_landmark_ar'],axis=1,inplace=True)


In [26]:
#DimNearestMetro Table
DimNearestMetro  = pd.DataFrame(staging_db['nearest_metro_en'].unique(),columns=['nearest_metro_en'])
DimNearestMetro['nearest_metro_ar']  = pd.DataFrame(staging_db['nearest_metro_ar'].unique())

DimNearestMetro['Nearest_metro_Id'] = pd.Series(range(len(DimNearestMetro)))
DimNearestMetro = DimNearestMetro[['Nearest_metro_Id', 'nearest_metro_en', 'nearest_metro_ar']]
DimNearestMetro

,Nearest_metro_Id,nearest_metro_en,nearest_metro_ar
0,0,Buj Khalifa Dubai Mall Metro Station,محطة مترو بوج خليفة دبي مول
1,1,Nakheel Metro Station,محطة مترو النخيل
2,2,NaN,NaN
3,3,Jumeirah Beach Residency,مساكن شاطئ جميرا
4,4,Rashidiya Metro Station,محطة مترو الراشدية
5,5,Ibn Battuta Metro Station,محطة مترو ابن بطوطة
6,6,Mina Seyahi,مينا السياحي
7,7,Damac Properties,عقارات داماك
8,8,Jumeirah Lakes Towers,أبراج بحيرات جميرا
9,9,Dubai Internet City,مدينة دبي للإنترنت


In [27]:
# Generate nearest_metro_Id_PY Colmn 
temp_mapper = {key:value for key,value in zip(DimNearestMetro['nearest_metro_en'],DimNearestMetro['Nearest_metro_Id'])}
staging_db['generated_nearest_metro_Id_PY'] = staging_db['nearest_metro_en'].map(temp_mapper)
staging_db.drop(['nearest_metro_en', 'nearest_metro_ar'],axis=1,inplace=True)


In [28]:
#DimNearestMall Table
DimNearestMall  = pd.DataFrame(staging_db['nearest_mall_en'].unique(),columns=['nearest_mall_en'])
DimNearestMall['nearest_mall_ar']  = pd.DataFrame(staging_db['nearest_mall_ar'].unique())

DimNearestMall['Nearest_mall_Id'] = pd.Series(range(len(DimNearestMall)))
DimNearestMall = DimNearestMall[['Nearest_mall_Id', 'nearest_mall_en', 'nearest_mall_ar']]
DimNearestMall

,Nearest_mall_Id,nearest_mall_en,nearest_mall_ar
0,0,Dubai Mall,مول دبي
1,1,Marina Mall,مارينا مول
2,2,NaN,NaN
3,3,City Centre Mirdif,سيتي سنتر مردف
4,4,Ibn-e-Battuta Mall,ابن بطوطة مول
5,5,Mall of the Emirates,مول الإمارات


In [29]:
# Generate nearest_metro_Id_PY Colmn 
temp_mapper = {key:value for key,value in zip(DimNearestMall['nearest_mall_en'],DimNearestMall['Nearest_mall_Id'])}
staging_db['generated_nearest_mall_Id_PY'] = staging_db['nearest_mall_en'].map(temp_mapper)
staging_db.drop(['nearest_mall_en', 'nearest_mall_ar'],axis=1,inplace=True)

In [ ]:
# tenant_type_id                                1.0
# tenant_type_ar                                شخص
# tenant_type_en                             Person

In [30]:
#DimTenant Table
DimTenant  = pd.DataFrame(staging_db['tenant_type_en'].unique(),columns=['tenant_type_en'])
DimTenant['tenant_type_ar']  = pd.DataFrame(staging_db['tenant_type_ar'].unique())

DimTenant['tenant_type_id'] = pd.Series(range(len(DimTenant)))
DimTenant = DimTenant[['tenant_type_id', 'tenant_type_en', 'tenant_type_ar']]
staging_db = staging_db.drop(['tenant_type_en','tenant_type_ar'], axis = 1)
DimTenant

,tenant_type_id,tenant_type_en,tenant_type_ar
0,0,Authority,جهة
1,1,Person,شخص
2,2,NaN,NaN


In [33]:
staging_db = staging_db[['contract_id', 'contract_reg_type_id','contract_start_date','contract_end_date', 'ejari_bus_property_type_id', 'ejari_property_type_id', 'ejari_property_sub_type_id', 'project_number', 'area_id', 'tenant_type_id', 'generated_property_Id_PY', 'generated_master_project_Id_PY', 'generated_nearest_landmark_Id_PY', 'generated_nearest_metro_Id_PY', 'generated_nearest_mall_Id_PY', 'contract_amount', 'annual_amount', 'no_of_prop', 'line_number', 'is_free_hold', 'actual_area']]

In [68]:
type(DimDate['contract_end_date'].iloc[50])

str

In [64]:
# staging_db.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/Rent_Contracts_Enc.csv',index=False)
DimDate.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimDate.csv',index=False)
DimContractType.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimContractType.csv',index=False)
DimEjaripropertytype.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimEjaripropertytype.csv',index=False)
DimEjaribuspropertytype.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimEjaribuspropertytype.csv',index=False)
DimEjaripropertysubtype.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimEjaripropertysubtype.csv',index=False)
DimProject.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimProject.csv',index=False)
DimArea.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimArea.csv',index=False)
DimTenant.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimTenant.csv',index=False)
DimPropertyUsage.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimPropertyUsage.csv',index=False)
DimMasterProject.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimMasterProject.csv',index=False)
DimNearestLandmark.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimNearestLandmark.csv',index=False)
DimNearestMall.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimNearestMall.csv',index=False)
DimNearestMetro.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/DimNearestMetro.csv',index=False)





In [2]:
Rent_Contracts_Enc = pd.read_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/Rent_Contracts_Enc.csv')


In [6]:
id_col = pd.Series(range(len(Rent_Contracts_Enc)))
Rent_Contracts_Enc['Fact_Table_ID'] = id_col
Rent_Contracts_Enc.to_csv('F:/Prjoects/03 Full Real State Dubai DLD/Registration/Staging and final/Rent_Contracts_Enc.csv',index=False)

In [5]:
Rent_Contracts_Enc

,contract_id,contract_reg_type_id,contract_start_date,contract_end_date,ejari_bus_property_type_id,ejari_property_type_id,ejari_property_sub_type_id,project_number,area_id,tenant_type_id,...,generated_nearest_landmark_Id_PY,generated_nearest_metro_Id_PY,generated_nearest_mall_Id_PY,contract_amount,annual_amount,no_of_prop,line_number,is_free_hold,actual_area,Fact_Table_ID
0,CRT1012981266,1,07-04-2019,06-04-2020,2,2.0,422.0,467.0,526,2.0,...,0,0,0,85000,85000,1,1,1,140.0,0
1,CRT1012983196,1,20-04-2019,19-04-2020,4,841.0,2.0,NaN,442,1.0,...,1,1,1,110000,110000,1,1,1,734.0,1
2,CRT1012984226,1,11-04-2019,10-04-2020,4,841.0,3.0,1488.0,506,1.0,...,2,2,2,100000,100000,1,1,1,324.0,2
3,CRT1012984996,2,18-03-2019,17-03-2020,4,841.0,3.0,1377.0,463,1.0,...,3,2,2,150000,150000,1,1,1,405.0,3
4,CRT1012986616,1,15-04-2019,14-04-2020,2,842.0,1.0,NaN,330,1.0,...,4,3,1,95000,95000,1,1,1,103.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6667408,CNT9995,1,01-12-2010,30-11-2011,2,842.0,4.0,NaN,234,1.0,...,8,28,3,81840,81840,1,1,0,260.0,6667408
6667409,CNT9996,1,01-07-2010,30-06-2011,2,842.0,11.0,NaN,234,1.0,...,8,28,3,28000,28000,1,1,0,26.0,6667409
6667410,CNT9997,1,01-09-2010,31-08-2011,2,842.0,11.0,NaN,234,1.0,...,8,28,3,28000,28000,1,1,0,42.0,6667410
6667411,CNT9998,1,01-08-2010,31-07-2011,2,842.0,11.0,NaN,234,1.0,...,8,28,3,30000,30000,1,1,0,36.0,6667411
